In [ ]:
from tensorflow.python.saved_model import tag_constants
import tensorflow as f

graph = tf.Graph()
with restored_graph.as_default()
    with tf.Session as sess:
        tf.saved_model.loader.load(
            sess,
            [tag_constants.SERVING],
            'logs/tf/runs/Em-300_FS-3-4-5_NF-300_D-0.75_L2-0.01_B-64_Ep-12_W2V-1_V-50000/checkpoints/model-7100.index'        
        )
    
        # Placeholders for input, output and dropout
        input_x = graph.get_tensor_by_name('input_x')
        input_y = graph.get_tensor_by_name('input_y')
        dropout_keep_prob = graph.get_tensor_by_name('dropout_keep_prob')
        
        scores = graph.get_tensor_by_name("scores")
        predictions = graph.get_tensor_by_name("predictions")
        
        sess.run(prediction, feed_dict={
            batch_size_placeholder: some_value,
            features_placeholder: some_other_value,
            labels_placeholder: another_value
        })